In [1]:
bucket='rekognitioncustomlabels-pvt'

In [2]:
#Copyright 2020 Amazon.com, Inc. or its affiliates. All Rights Reserved.
#PDX-License-Identifier: MIT-0 (For details, see https://github.com/awsdocs/amazon-rekognition-custom-labels-developer-guide/blob/master/LICENSE-SAMPLECODE.)

import boto3

def start_model(project_arn, model_arn, version_name, min_inference_units):

    client=boto3.client('rekognition')

    try:
        # Start the model
        print('Starting model: ' + model_arn)
        response=client.start_project_version(ProjectVersionArn=model_arn, MinInferenceUnits=min_inference_units)
        # Wait for the model to be in the running state
        project_version_running_waiter = client.get_waiter('project_version_running')
        project_version_running_waiter.wait(ProjectArn=project_arn, VersionNames=[version_name])

        #Get the running status
        describe_response=client.describe_project_versions(ProjectArn=project_arn,
            VersionNames=[version_name])
        for model in describe_response['ProjectVersionDescriptions']:
            print("Status: " + model['Status'])
            print("Message: " + model['StatusMessage']) 
    except Exception as e:
        print(e)
        
    print('Done...')
    
def main():
    project_arn='arn:aws:rekognition:us-west-2:406078665760:project/SmartStream-evm-custom-label-Dec2021/1640107420943'
    model_arn='arn:aws:rekognition:us-west-2:406078665760:project/SmartStream-evm-custom-label-Dec2021/version/SmartStream-evm-custom-label-Dec2021.2021-12-21T13.46.11/1640119571232'
    min_inference_units=1 
    version_name='SmartStream-evm-custom-label-Dec2021.2021-12-21T13.46.11'
    start_model(project_arn, model_arn, version_name, min_inference_units)

if __name__ == "__main__":
    main()

Starting model: arn:aws:rekognition:us-west-2:406078665760:project/SmartStream-evm-custom-label-Dec2021/version/SmartStream-evm-custom-label-Dec2021.2021-12-21T13.46.11/1640119571232
Status: RUNNING
Message: The model is running.
Done...


In [10]:
#Copyright 2020 Amazon.com, Inc. or its affiliates. All Rights Reserved.
#PDX-License-Identifier: MIT-0 (For details, see https://github.com/awsdocs/amazon-rekognition-custom-labels-developer-guide/blob/master/LICENSE-SAMPLECODE.)

import json
import boto3
import cv2
import math
import io

def analyzeVideo():
    videoFile = "Evm_alert_3.mp4"
    projectVersionArn = 'arn:aws:rekognition:us-west-2:406078665760:project/SmartStream-evm-custom-label-Dec2021/version/SmartStream-evm-custom-label-Dec2021.2021-12-21T13.46.11/1640119571232'

    rekognition = boto3.client('rekognition')        
    customLabels = []    
    cap = cv2.VideoCapture(videoFile)
    frameRate = cap.get(5) #frame rate
    while(cap.isOpened()):
        frameId = cap.get(1) #current frame number
        print("Processing frame id: {}".format(frameId))
        ret, frame = cap.read()
        if (ret != True):
            break
        if (frameId % math.floor(frameRate) == 0):
            hasFrame, imageBytes = cv2.imencode(".jpg", frame)

            if(hasFrame):
                response = rekognition.detect_custom_labels(
                    Image={
                        'Bytes': imageBytes.tobytes(),
                    },
                    ProjectVersionArn = projectVersionArn
                )
            
            for elabel in response["CustomLabels"]:
                elabel["Timestamp"] = (frameId/frameRate)*1000
                customLabels.append(elabel)
    
    # use red for alert and blue for ok
    #if (customLabels['Name'] == "evm_alert"):
    #        print("evm_alert")
    #else:
    #        print("evm_ok")
                
    #print(customLabels)

    with open(videoFile + ".json", "w") as f:
        f.write(json.dumps(customLabels)) 

    cap.release()



In [11]:
analyzeVideo()

In [ ]:
# stop the model so you don't get charged
import boto3
import time


def stop_model(model_arn):

    client=boto3.client('rekognition')

    print('Stopping model:' + model_arn)

    #Stop the model
    try:
        response=client.stop_project_version(ProjectVersionArn=model_arn)
        status=response['Status']
        print ('Status: ' + status)
    except Exception as e:  
        print(e)  

    print('Done...')
    
def main():
    
    model_arn='arn:aws:rekognition:us-west-2:406078665760:project/SmartStream-evm-custom-label-Dec2021/version/SmartStream-evm-custom-label-Dec2021.2021-12-21T13.46.11/1640119571232'
    stop_model(model_arn)

if __name__ == "__main__":
    main() 